<a href="https://colab.research.google.com/github/halaaab/IT_326-Project-Group-1/blob/main/Phase3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[1] Problem**

Breast cancer is one of the most common cancers affecting women worldwide. Early and accurate diagnosis is crucial for improving treatment outcomes and patient survival rates.
This project aims to analyze a dataset containing features computed from digitized images of fine-needle aspirates (FNA) of breast masses. By applying classification and clustering techniques, we aim to predict whether a tumor is malignant or benign, and to explore patterns and similarities among cases to gain deeper insights into the characteristics of cancerous and non-cancerous cells.